In [2]:
#Importing all necessary libraries for this project.
import pandas as pd
import zipfile
import kaggle

OSError: Could not find kaggle.json. Make sure it's located in C:\Users\tommy\.kaggle. Or use the environment method.

In [ ]:
#Downloading the dataset from Kaggle through its API.
!kaggle datasets download -d hmavrodiev/london-bike-sharing-dataset

london-bike-sharing-dataset.zip: Skipping, found more recently modified local copy (use --force to force download)


In [ ]:
#Extracting the data file from the zip file.
zipfile_name = 'london-bike-sharing-dataset.zip'
with zipfile.ZipFile(zipfile_name, 'r') as file:
    file.extractall()

In [ ]:
#Making a variable for the csv file through pandas.
bikes = pd.read_csv('london_merged.csv')

In [ ]:
#Looking at general information about the data.
#Some things to note about the info from the general information:
    #1: timestamp is an object, cnt(count) is an intiger, and everything else is a float variable.
    #2: weather_code, is_holiday, is_weekend, and season are all float variables. I assume this the case for ease of collecting the data, but it should be changed to string for easier analysis.
bikes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17414 entries, 0 to 17413
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   timestamp     17414 non-null  object 
 1   cnt           17414 non-null  int64  
 2   t1            17414 non-null  float64
 3   t2            17414 non-null  float64
 4   hum           17414 non-null  float64
 5   wind_speed    17414 non-null  float64
 6   weather_code  17414 non-null  float64
 7   is_holiday    17414 non-null  float64
 8   is_weekend    17414 non-null  float64
 9   season        17414 non-null  float64
dtypes: float64(8), int64(1), object(1)
memory usage: 1.3+ MB


In [ ]:
#Looking at how many rows/columns there are.
bikes.shape

(17414, 10)

In [ ]:
#Looking at the actual table with the data. Want to get an idea of what the data looks like so I know what I need to modify for coding/analysis purposes.
#After looking at this table I see a few things:
    #1: Column names are not ideal for analysis. Even though it is clear to me what each column is representing, it may not be for everybody. To increase inter-rater reliability I will change the names of the columns to something more clear.
    #2: The humidity values need to be changed to a number between 0 and 1 to represent the percent humidity.
bikes

,timestamp,cnt,t1,t2,hum,wind_speed,weather_code,is_holiday,is_weekend,season
0,2015-01-04 00:00:00,182,3.0,2.0,93.0,6.0,3.0,0.0,1.0,3.0
1,2015-01-04 01:00:00,138,3.0,2.5,93.0,5.0,1.0,0.0,1.0,3.0
2,2015-01-04 02:00:00,134,2.5,2.5,96.5,0.0,1.0,0.0,1.0,3.0
3,2015-01-04 03:00:00,72,2.0,2.0,100.0,0.0,1.0,0.0,1.0,3.0
4,2015-01-04 04:00:00,47,2.0,0.0,93.0,6.5,1.0,0.0,1.0,3.0
...,...,...,...,...,...,...,...,...,...,...
17409,2017-01-03 19:00:00,1042,5.0,1.0,81.0,19.0,3.0,0.0,0.0,3.0
17410,2017-01-03 20:00:00,541,5.0,1.0,81.0,21.0,4.0,0.0,0.0,3.0
17411,2017-01-03 21:00:00,337,5.5,1.5,78.5,24.0,4.0,0.0,0.0,3.0
17412,2017-01-03 22:00:00,224,5.5,1.5,76.0,23.0,4.0,0.0,0.0,3.0


In [ ]:
#Here I am confirming that the weather code values are consistent with the key provided with the dataset. Everything seems to be in order.
bikes.weather_code.value_counts()

weather_code
1.0     6150
2.0     4034
3.0     3551
7.0     2141
4.0     1464
26.0      60
10.0      14
Name: count, dtype: int64

In [ ]:
#Here I am checking to see that the season values are also in order with the key provided with the dataset. Everything looks good.
bikes.season.value_counts()

season
0.0    4394
1.0    4387
3.0    4330
2.0    4303
Name: count, dtype: int64

In [ ]:
#I am making new column names that are less ambiguous.
new_cols ={
    'timestamp':'time',
    'cnt':'count',
    't1':'real_temp_c',
    't2':'feels_like_temp_c',
    'hum':'humidity',
    'wind_speed':'wind_speed_kph',
    'weather_code':'weather',
    'is_holiday':'is_holiday',
    'is_weekend':'is_weekend',
    'season':'season'
}

#renaming the columns with the new column names.
bikes.rename(new_cols, axis=1, inplace=True)

In [ ]:
#Converting humidity values to percent values (between 0 and 1).
bikes.humidity = bikes.humidity / 100

In [ ]:
#Replacing season code values with the actual season name.
season_dict ={
    '0.0':'spring',
    '1.0':'summer',
    '2.0':'fall',
    '3.0':'winter'
}

#Replacing the weather code values to the names of the weather given by the key provided.
weather_dict ={
    '1.0':'clear',
    '2.0':'few_clouds',
    '3.0':'broken_clouds',
    '7.0':'light_rain',
    '4.0':'cloudy',
    '10.0':'rain_with_thunderstorm',
    '26.0':'snowfall'
}

#Converting season values from float to string and then mapping new values to the dataset.
bikes.season = bikes.season.astype('str')
bikes.season = bikes.season.map(season_dict)

#Converting weather values from float to string and then mapping new values to the dataset.
bikes.weather = bikes.weather.astype('str')
bikes.weather = bikes.weather.map(weather_dict)

In [ ]:
#Replacing is_holiday and is_weekend values from integers to strings of Y/N

is_holiday_dict ={
    '0.0':'N',
    '1.0':'Y'
}

is_weekend_dict ={
    '0.0':'N',
    '1.0':'Y'
}

#Converting values from int to str and mapping new values to dataset.
bikes.is_holiday = bikes.is_holiday.astype('str')
bikes.is_holiday = bikes.is_holiday.map(is_holiday_dict)

bikes.is_weekend = bikes.is_weekend.astype('str')
bikes.is_weekend = bikes.is_weekend.map(is_weekend_dict)

In [ ]:
#Checking to see that everything is in order and as I want it.

bikes.head(10)

,time,count,real_temp_c,feels_like_temp_c,humidity,wind_speed_kph,weather,is_holiday,is_weekend,season
0,2015-01-04 00:00:00,182,3.0,2.0,0.930,6.0,broken_clouds,N,Y,winter
1,2015-01-04 01:00:00,138,3.0,2.5,0.930,5.0,clear,N,Y,winter
2,2015-01-04 02:00:00,134,2.5,2.5,0.965,0.0,clear,N,Y,winter
3,2015-01-04 03:00:00,72,2.0,2.0,1.000,0.0,clear,N,Y,winter
4,2015-01-04 04:00:00,47,2.0,0.0,0.930,6.5,clear,N,Y,winter
5,2015-01-04 05:00:00,46,2.0,2.0,0.930,4.0,clear,N,Y,winter
6,2015-01-04 06:00:00,51,1.0,-1.0,1.000,7.0,cloudy,N,Y,winter
7,2015-01-04 07:00:00,75,1.0,-1.0,1.000,7.0,cloudy,N,Y,winter
8,2015-01-04 08:00:00,131,1.5,-1.0,0.965,8.0,cloudy,N,Y,winter
9,2015-01-04 09:00:00,301,2.0,-0.5,1.000,9.0,broken_clouds,N,Y,winter


In [ ]:
#Extracting modified table to an Excel file for analysis and visualization.
bikes.to_excel('london_bikes_final.xlsx', sheet_name='Data')